In [3]:
from langchain.chat_models import ChatOpenAI
# promptTemplate은 그냥 string을 이용해서 template을 만들고 ChatPromptTemplate은 template을 message로 부터 만든다.
from langchain.prompts import PromptTemplate

#값이 높아질수록 더 많은 창의성과 무작위성을 갖게된다.
chat = ChatOpenAI(temperature=0.1)

#단순 텍스트 predict
chat.predict("...")

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}.")

# template 위쪽에는 개발자들에게 필요한 일종의 검증계층이 있어 실수를 줄일 수 있음
#template을 format 하여 Prompt에 저장
prompt = template.format(country_a="Mexico", country_b="Thailand" )

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage




#SystemMessage is sent by the system, the AIMessage is sent by the AI and the HumanMessage is sent by the user. They all make up a conversation.
message=[
  #systemMessage-AI에 일종의 기본 설정, 기본 값, 기본 context 설정, LLM에 설정들을 제공하기 위한 메세지
  SystemMessage(content="You are a geography expert, And you only reply in Italian"),
  #이미 존재하는 가상 대화 설정, AI가 답변을 한것으로 가정
  #값을 메모리에 추가
  AIMessage(content="Ciao, michiamo Palolo!"),
  #사용자로서 질문
  HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

# predict message
chat.predict_messages(message)


In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI


chat = ChatOpenAI(temperature=0.1)

#more customizable
template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert, And you only reply in {language}"),
  ("ai", "Ciao, michiamo {name}!"),
  ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

#validate variables
prompt = template.format_messages(language="Greek", name="Socrates", country_a="Mexico", country_b="Thailand")

chat.predict_messages(prompt)



AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')